In [4]:
import pandas as pd
from glob import glob
import numpy as np
from aggmap import AggMap, AggModel

# small smaple size data tends to generate random fmaps

In [5]:
flist = glob('../data/*.csv.gzip')
flist = pd.Series(flist).sort_values().tolist()

In [7]:
flist = ['../data/05_CHOL.csv.gzip']

In [8]:

fall = []
for i in flist:
    df1 = pd.read_csv(i, compression='gzip', index_col = 0)
    df1['class'] = i.split('/')[-1].split('.csv')[0]
    fall.append(df1)

In [9]:
df = pd.concat(fall, axis=0)

In [10]:
df.groupby('class').size().to_frame(name = 'count').to_csv('classes.csv')

In [11]:
dfx = df[df.columns[:-1]]
dfy = df[df.columns[-1:]]

In [12]:
dfx = np.log2(dfx + 1) #apply log2(x + 1)

In [ ]:
mp = AggMap(dfx)

2021-03-15 10:49:28,452 - INFO - [bidd-aggmap] - Calculating distance ...
2021-03-15 10:49:35,287 - INFO - [bidd-aggmap] - the number of process is 16


 42%|####2     | 22857914/53877390 [12:17<16:28, 31369.06it/s] 

In [ ]:
mp.fit(cluster_channels = 5, verbose = 0)

In [25]:
mp.plot_scatter(htmlname = 'low-sample')

2021-03-15 15:48:44,191 - INFO - [bidd-aggmap] - generate file: ./low-sample_feature points_10381_correlation_umap_scatter
2021-03-15 15:48:44,509 - INFO - [bidd-aggmap] - save html file to ./low-sample_feature points_10381_correlation_umap_scatter


In [24]:
mp.plot_grid(htmlname = 'low-sample')

2021-03-15 15:48:33,172 - INFO - [bidd-aggmap] - generate file: ./low-sample_feature points_10381_correlation_umap_mp
2021-03-15 15:48:33,448 - INFO - [bidd-aggmap] - save html file to ./low-sample_feature points_10381_correlation_umap_mp


In [23]:
mp.fmap_shape

(102, 102)

In [31]:
import os
from aggmap import loadmap
from joblib import dump,load

data_save_folder = '/raid/shenwanxiang/transcriptome/pan-cancer/very-low-sample'

flist = glob('../data/*.csv.gzip')
flist = pd.Series(flist).sort_values().tolist()
fall = []
for i in flist:
    df1 = pd.read_csv(i, compression='gzip', index_col = 0)
    df1['class'] = i.split('/')[-1].split('.csv')[0]
    fall.append(df1)
df = pd.concat(fall, axis=0)
dfx = df[df.columns[:-1]]
dfx = np.log2(dfx + 1) #apply log2(x + 1)

scale_method = 'minmax'
mp_larger = loadmap('/raid/shenwanxiang/agg_mp_object/pan-cancer.mp')
mp.x_min = mp_larger.x_min
mp.x_max = mp_larger.x_max




if not os.path.exists(data_save_folder):
    os.makedirs(data_save_folder)
    
if not os.path.exists(os.path.join(data_save_folder, 'Agg5.data')):
    # save multi-channel
    X = mp.batch_transform(dfx.values, scale_method = scale_method) 
    n, w, h, c = X.shape
    dump(X,  os.path.join(data_save_folder, 'Agg5.data')) 
    
    ## save single channel
    X1 = X.sum(axis=-1).reshape(n, w, h, 1)
    dump(X1,  os.path.join(data_save_folder, 'Agg1.data')) 

else:
    X = load(os.path.join(data_save_folder, 'Agg5.data'))
    
    
mp.save(os.path.join(data_save_folder, 'very_low.mp'))

100%|##########| 10446/10446 [30:24<00:00,  5.73it/s]


['/raid/shenwanxiang/transcriptome/pan-cancer/very-low-sample/very_low.mp']